### import requirements

In [74]:
import requests, pandas as pd
from bs4 import BeautifulSoup

### scrape contents of article "List of postal codes of Canada"

In [75]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page, 'html.parser')

### parse table data for postal code data

In [76]:
table = soup.find('table', attrs={'class': 'wikitable sortable'})
postal_code_data = table.find_all('tr')

### place postal code, borough, and neighborhood into dataframe

In [77]:
postal_codes = []

for row in results:
    contents = row.findAll('td')
    if len(contents) > 0: 
        postal_codes.append([char.text.strip('\n') for char in contents])
    

df = pd.DataFrame(postal_codes, columns=['PostalCode', 'Borough', 'Neighbourhood']).reset_index(drop=True)
df = df.loc[df.Borough.ne('Not assigned')].reset_index(drop=True)
df_final = df.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna()))).reset_index(drop=True)

print(df_final.head(10))
print(df_final.shape)

  PostalCode      Borough                                    Neighbourhood
0        M1B  Scarborough                                   Malvern, Rouge
1        M1C  Scarborough           Rouge Hill, Highland Creek, Port Union
2        M1E  Scarborough                West Hill, Guildwood, Morningside
3        M1G  Scarborough                                           Woburn
4        M1H  Scarborough                                        Cedarbrae
5        M1J  Scarborough                              Scarborough Village
6        M1K  Scarborough      Kennedy Park, East Birchmount Park, Ionview
7        M1L  Scarborough                  Oakridge, Clairlea, Golden Mile
8        M1M  Scarborough  Cliffside, Scarborough Village West, Cliffcrest
9        M1N  Scarborough                      Cliffside West, Birch Cliff
(103, 3)


### import coordinates file

In [78]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

### map coordinates to postal codes

In [79]:
df_final['Latitude'] = df_final['PostalCode'].map(coordinates.set_index('Postal Code')['Latitude']).fillna(0)
df_final['Longitude'] = df_final['PostalCode'].map(coordinates.set_index('Postal Code')['Longitude']).fillna(0)
df_final.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Guildwood, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


### import requirements for neighborhood exploration using FourSquare 

In [80]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests
import pandas as pd
import numpy as np
import random

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.10 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  31.35 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.76 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.35 MB/s
vincent-0.4.4- 100% |###################

### define FourSquare credentials

In [83]:
CLIENT_ID = '###################################'
CLIENT_SECRET = '##################################'
VERSION = '20180604'
LIMIT = 30

### set location to first postal code in dataframe

In [88]:
geolocator = Nominatim(user_agent="capstone-project")
latitude = df_final['Latitude'].iat[0]
longitude = df_final['Longitude'].iat[0]
print(latitude, longitude)

43.8066863 -79.1943534


### search for Italian food within 500 meters of coordinates above

In [92]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
print(url)

results = requests.get(url).json()
print(results)

venues = results['response']['venues']

dataframe = json_normalize(venues)
print(dataframe.head())



Italian .... OK!
https://api.foursquare.com/v2/venues/search?client_id=FMJGT45YLSLQDT2UT2KTIVKUCE4ONL3XMP13MZRICUA21NKJ&client_secret=FEMR4C4ZW2C3TONPAJKM3VWKPSRHYDWHYPGY41MB3BH1SZOP&ll=43.806686299999996,-79.19435340000001&v=20180604&query=Italian&radius=500&limit=30
{'meta': {'code': 200, 'requestId': '5c243b521ed2194bd6358f57'}, 'response': {'venues': []}}
Empty DataFrame
Columns: []
Index: []


### map venues at coordinates above

In [94]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


# display map
venues_map